<a href="https://colab.research.google.com/github/defeng1/asl-assistant/blob/back-end/model_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pathlib
from tensorflow.keras import layers
from google.colab import drive
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#Parameters
img_height = 200
img_width = 200

num_classes = 29
num_samples = 1024
batch_size = 32

In [10]:
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01):
	# create model
	model = tf.keras.Sequential([
		layers.experimental.preprocessing.Rescaling(1./255),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Flatten(),
		layers.Dense(128, activation='relu'),
		layers.Dense(num_classes)
	])
 
	# Compile model
	adam = tf.keras.optimizers.Adam(learning_rate=learn_rate)
	model.compile(
		optimizer = adam,
		loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
		metrics=['accuracy'])
	
	return model

In [6]:
#Importing the dataset
data_dir = "/content/gdrive/My Drive/Capstone/Dataset/test_set"
data_dir = pathlib.Path(data_dir)

ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  image_size=(img_height, img_width))

Found 8702 files belonging to 29 classes.


In [8]:
images = np.zeros(shape=(num_samples, img_height, img_width, 3))
labels = np.zeros(shape=(num_samples))

i = 0
for images_batch, labels_batch in ds.as_numpy_iterator():
  images[i*batch_size : (i+1)*batch_size] = images_batch
  labels[i*batch_size : (i+1)*batch_size] = labels_batch
  print("Loading batch {} done".format(i))
  if (i + 1) * batch_size == num_samples:
    print("Finished loading samples for grid search")
    break
  i += 1

Loading batch 0 done
Loading batch 1 done
Loading batch 2 done
Loading batch 3 done
Loading batch 4 done
Loading batch 5 done
Loading batch 6 done
Loading batch 7 done
Loading batch 8 done
Loading batch 9 done
Loading batch 10 done
Loading batch 11 done
Loading batch 12 done
Loading batch 13 done
Loading batch 14 done
Loading batch 15 done
Loading batch 16 done
Loading batch 17 done
Loading batch 18 done
Loading batch 19 done
Loading batch 20 done
Loading batch 21 done
Loading batch 22 done
Loading batch 23 done
Loading batch 24 done
Loading batch 25 done
Loading batch 26 done
Loading batch 27 done
Loading batch 28 done
Loading batch 29 done
Loading batch 30 done
Loading batch 31 done
Finished loading samples for grid search


In [11]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
batch_sizes = [10, 20, 40, 60, 80, 100]
epochs = [1, 5, 10, 15]
param_grid = dict(batch_size=batch_sizes, epochs=epochs, learn_rate=learn_rate)

# grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(images, labels)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored